In [1]:
from seagul.envs.simple_nonlinear import TreeSimple, TreeMulti
import matplotlib.pyplot as plt
from seagul.mesh import BoxMesh
import numpy as np
from itertools import product
from matplotlib import cm
%matplotlib tk

/home/sgillen/work/seagul/seagul/envs/__init__.py:52: UserWarning: Warning, pybullet envs not installed
  warnings.warn("Warning, pybullet envs not installed")


pygame 2.0.1 (SDL 2.0.14, Python 3.6.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
env = TreeMulti(dt = .1)
env.reset()

/home/sgillen/anaconda3/envs/baselines/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


array([4.75412401, 2.        ])

In [3]:
du = .1
U = np.arange(-env.L,env.L,du)
X = np.arange(-1,11,du*env.dt)
Y = np.arange(3,-2,env.g*env.dt)

XY = list(product(X,Y))
V = BoxMesh(du*env.dt) 
Vu = BoxMesh(du*env.dt) 


for xy in XY:
    V[xy] = 0
    Vu[xy] = 0


In [4]:
#V[s1] = np.max(r(s1,u) + V[s2])
for outer_loop in range(10):
    for s1 in XY:
        Vn_list = []
        for u in U:
            env.reset()
            env.X = s1
            s2, r, d, _ = env.step(u)
            s2 = tuple(s2.tolist())
            
            if d is True:
                Vn_list.append(r)
            elif s2 in V:
                Vn_list.append(V[s2] + r)
            else:
                if (s1[1] > 0):
                    print(f"{s1}->{s2} (u = {u}) not in mesh, skipping")

        if len(Vn_list):
            V[s1] = np.max(Vn_list)
            Vu[s1]= U[np.argmax(Vn_list)] 
        else:pass
    
    xx,yy = np.meshgrid(X,Y,indexing='ij')
    uu = np.zeros_like(xx)
    for i,x in enumerate(X):
        for j,y in enumerate(Y):
            uu[i,j] = Vu[(x,y)] 


    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    ax.plot_surface(xx,yy,uu, facecolors=cm.Spectral(uu/np.amax(uu)), alpha=.6)
    ax.set_title(str(outer_loop))




In [5]:
#V[s1] = np.max(r(s1,u) + V[s2])

# def env_worker(s1) -> s2, r, d

# for outer_loop in range(10):
#     for s1 in XY:
#         Vn_list = []
#         for u in U:
#             env.reset()
#             env.X = s1
#             s2, r, d, _ = env.step(u)
#             s2 = tuple(s2.tolist())
            
#             if d is True:
#                 Vn_list.append(r)
#             elif s2 in V:
#                 Vn_list.append(V[s2] + r)
#             else:
#                 if (s1[1] > 0):
#                     print(f"{s1}->{s2} (u = {u}) not in mesh, skipping")

#         if len(Vn_list):
#             V[s1] = np.max(Vn_list)
#             Vu[s1]= U[np.argmax(Vn_list)] 
#         else:pass
    
#     xx,yy = np.meshgrid(X,Y,indexing='ij')
#     uu = np.zeros_like(xx)
#     for i,x in enumerate(X):
#         for j,y in enumerate(Y):
#             uu[i,j] = Vu[(x,y)] 


#     fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
#     ax.plot_surface(xx,yy,uu, facecolors=cm.Spectral(uu/np.amax(uu)), alpha=.6)
#     ax.set_title(str(outer_loop))


